Caso 1:
Imagine que ud es un data scientist y le piden que realice un modelo de clasificación binaria utilizando solamente una técnica, dada la urgencia. Si eliges este caso, a continuación se comparte un dataset en el siguiente link y responda las preguntas.

In [8]:
import pandas as pd

#Primero que todo cargo mi archivo dataset_Caso_1.csv
df = pd.read_csv("dataset_Caso_1.csv")

#Luego visualizo adecuadamente cada tipo de datos que contiene cada columna viendo el encabezado de las mismas
print(df.head())

#primero que todo nos cercioramos de que en efecto, como nos dice el enunciado, la columna
#target solamente contenga dos valores (es decir, es una variable binaria)
# df['target'][df['target']==1]

#por último imprimo también las propia dataframe
df


#vemos además la cantidad de valores diferentes en las columnas que contienen variables categóricas
# Contar valores únicos en las columnas x3 y x4
valores_x3 = df["x3"].nunique()
valores_x4 = df["x4"].nunique()

print(f"Cantidad de valores diferentes en x3: {valores_x3}")
print(f"Cantidad de valores diferentes en x4: {valores_x4}")


         x1        x2   x3     x4  x5  x6  x7   x8  target
0   50.1341 -326.0000  SAT  MZBER   0   0   6 -6.5       0
1   50.1341 -326.0000  SAT  MZBER   0   0   6 -4.5       0
2  124.3276 -275.1935  LCV  MZBER   0   0   3 -2.5       0
3   50.1341 -326.0000  SAT  MZBER   0   0   3 -4.5       0
4   85.3905 -298.8632  XJB  MZBER   0   0   5 -4.5       0
Cantidad de valores diferentes en x3: 4
Cantidad de valores diferentes en x4: 4



A continuación hago una búsqueda del mejor modelo de entre arboles de decision, random forests y redes neronales, cada una con sus mejores hiperparámetros

In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score, f1_score

df_completo = df

caracteristicas_modelo = df_completo.drop(columns=["target", "x3", "x4"])
objetivo_modelo = df_completo["target"]

X_entrenamiento, X_prueba, y_entrenamiento, y_prueba = train_test_split(
    caracteristicas_modelo, objetivo_modelo, test_size=0.2, random_state=42, stratify=objetivo_modelo
)

datos_entrenamiento = X_entrenamiento.copy()
datos_entrenamiento["objetivo"] = y_entrenamiento

datos_minoria = datos_entrenamiento[datos_entrenamiento["objetivo"] == 1]
datos_mayoria = datos_entrenamiento[datos_entrenamiento["objetivo"] == 0]

datos_minoria_oversample = datos_minoria.sample(n=len(datos_mayoria), replace=True, random_state=42)
datos_entrenamiento_balanceado = pd.concat([datos_mayoria, datos_minoria_oversample])
datos_entrenamiento_balanceado = datos_entrenamiento_balanceado.sample(frac=1, random_state=42).reset_index(drop=True)

X_entrenamiento_balanceado = datos_entrenamiento_balanceado.drop(columns=["objetivo"])
y_entrenamiento_balanceado = datos_entrenamiento_balanceado["objetivo"]

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

parametros_dt = {
    "max_depth": [None, 10, 20, 30],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}

parametros_rf = {
    "n_estimators": np.arange(50, 500, 50),
    "max_depth": [None, 10, 20, 30, 40],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "bootstrap": [True, False]
}

parametros_nn = {
    "hidden_layer_sizes": [(50,), (100,), (50, 50), (100, 100)],
    "activation": ["relu", "tanh"],
    "alpha": [0.0001, 0.001, 0.01],
    "solver": ["adam", "sgd"],
    "learning_rate": ["constant", "adaptive"]
}

def entrenar_modelo(modelo, parametros):
    busqueda = RandomizedSearchCV(modelo, parametros, n_iter=20, cv=cv, scoring="roc_auc", random_state=42, n_jobs=-1)
    busqueda.fit(X_entrenamiento_balanceado, y_entrenamiento_balanceado)
    mejor_modelo = busqueda.best_estimator_
    
    pred_entrenamiento = mejor_modelo.predict_proba(X_entrenamiento_balanceado)[:, 1]
    pred_prueba = mejor_modelo.predict_proba(X_prueba)[:, 1]

    auc_entrenamiento = roc_auc_score(y_entrenamiento_balanceado, pred_entrenamiento)
    auc_prueba = roc_auc_score(y_prueba, pred_prueba)

    pred_clase_prueba = (pred_prueba >= 0.5).astype(int)
    f1_prueba = f1_score(y_prueba, pred_clase_prueba, zero_division=0)

    return mejor_modelo, busqueda.best_params_, auc_entrenamiento, auc_prueba, f1_prueba

modelo_dt, params_dt, auc_train_dt, auc_test_dt, f1_dt = entrenar_modelo(
    DecisionTreeClassifier(class_weight="balanced", random_state=42), parametros_dt
)

modelo_rf, params_rf, auc_train_rf, auc_test_rf, f1_rf = entrenar_modelo(
    RandomForestClassifier(class_weight="balanced", random_state=42), parametros_rf
)

modelo_nn, params_nn, auc_train_nn, auc_test_nn, f1_nn = entrenar_modelo(
    MLPClassifier(max_iter=500, random_state=42), parametros_nn
)

resultados = {
    "Decision Tree": {"AUC Entrenamiento": auc_train_dt, "AUC Prueba": auc_test_dt, "F1 Prueba": f1_dt, "Hiperparámetros": params_dt},
    "Random Forest": {"AUC Entrenamiento": auc_train_rf, "AUC Prueba": auc_test_rf, "F1 Prueba": f1_rf, "Hiperparámetros": params_rf},
    "Neural Network": {"AUC Entrenamiento": auc_train_nn, "AUC Prueba": auc_test_nn, "F1 Prueba": f1_nn, "Hiperparámetros": params_nn},
}

mejor_modelo = max(resultados, key=lambda x: resultados[x]["AUC Prueba"])

In [21]:
print("\nResultados:")

for modelo, info in resultados.items():
    print(f"\n{modelo}:")
    print(f"  AUC Entrenamiento: {info['AUC Entrenamiento']:.4f}")
    print(f"  AUC Prueba: {info['AUC Prueba']:.4f}")
    print(f"  F1 Score Prueba: {info['F1 Prueba']:.4f}")
    print(f"  Mejores hiperparámetros: {info['Hiperparámetros']}")

print(f"\n🏆 Mejor modelo: {mejor_modelo} con AUC Prueba de {resultados[mejor_modelo]['AUC Prueba']:.4f} y F1-score de {resultados[mejor_modelo]['F1 Prueba']:.4f}")


Resultados:

Decision Tree:
  AUC Entrenamiento: 1.0000
  AUC Prueba: 0.4975
  F1 Score Prueba: 0.0000
  Mejores hiperparámetros: {'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': None}

Random Forest:
  AUC Entrenamiento: 1.0000
  AUC Prueba: 0.5442
  F1 Score Prueba: 0.0000
  Mejores hiperparámetros: {'n_estimators': 250, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 30, 'bootstrap': True}

Neural Network:
  AUC Entrenamiento: 0.9700
  AUC Prueba: 0.5770
  F1 Score Prueba: 0.0702
  Mejores hiperparámetros: {'solver': 'adam', 'learning_rate': 'constant', 'hidden_layer_sizes': (50,), 'alpha': 0.001, 'activation': 'tanh'}

🏆 Mejor modelo: Neural Network con AUC Prueba de 0.5770 y F1-score de 0.0702


Utilicé una grilla GridSearch para encontrar los mejores hiperparámetros para tres modelos de clasificación diferentes: decision tree, random forest y multi perceptron layer (neural network/redes neuronales) y encontré que el mejor de ellos, es decir, el que arrojaba un valor más alto de f1-score fue la red neuronal de hiperparámetros  {'solver': 'adam', 'learning_rate': 'constant', 'hidden_layer_sizes': (50,), 'alpha': 0.001, 'activation': 'tanh'}, donde evidentemente el f1-score debe ser lo mayor posible para poder decir que el modelo clasifica bien las dos clases. Por otro lado, intenté hacer un oversampling y convertir las columnas x3 y x4 a variables dummies (es decir, crear variables binarias a partir de variables categoricas como aquellas), pero el f1-score se mantenia en 0 para el random forest. Por otro lado, si bien un AUC  de 57 por ciento es casi igual a un clasificador que clasifica como por azar (aleatoriamente, cuyo AUC sería de 50 por ciento), sin embargo, es preferible tener un f1-score mayor. Intenté (fracasando) hacer un oversampling sobre la variable binaria de la columna target porque existía un desbalance totalmente absurdo, como un 1 por cada 2000 0's. Evidentemente, utilizar aquí Stratify no sirve de mucho, ni tampoco lo fue el uso de balance de clases en random forest ni en decision tree. Por lo que me quedaría con el MPL (multi-perceptron layer, o multiples capas de perceptrones o red neuronal) para la clasificación de este set de datos. 